sources:

https://crypto.stackexchange.com/questions/6713/low-public-exponent-attack-for-rsa

https://stackoverflow.com/questions/55436001/cube-root-of-a-very-large-number-using-only-math-library

In [1]:
from Crypto.PublicKey import RSA
import base64

In [2]:
key0 = RSA.import_key(open("clef0_pub.pem").read())
key1 = RSA.import_key(open("clef1_pub.pem").read())
key2 = RSA.import_key(open("clef2_pub.pem").read())

In [6]:
m0 = "vjXJvWis95tc25G+wxC5agClCJFB9vUslFyV+I4bSiwS4Sm6k8eF61EizKo4hZFwROlO3Ci3YQaTrAm+Y9/qEbM7asvwKTePKX+cLVN61l0xxfTL8CdoXkRE2rSczp1AzzmFz83OHgszX/Wf7kgWU4M73+efPvU9FmcWOauakrdJZx8B0ErJ5cYWNS0ZCam0Nlz+pISqdSJ6MSz0Ek62Ulb3ei8I41FOdHtd8mhC7dfpdfmVLOSEW4yEnn5iuMW0ydvW055dodLc9RKcvJafH9e3zf8/S1/RORXZoUHWnBXBEFOl8iVXz70GcDTPzIhxh6imi0ynLbV68qW2vw2XRQ=="
m1 = "Mi4MnobVabt1+Q7R/0aIYBBpeRxPRuR6gkhr/Wbw/D23ywu2KbUYBab2XbEguRz8yzBlxScbFjjb98DuILxURoFN8lNKVJLS3d/IrGGr0hjocbz27uBS97hseX0S4nd+BL6LUn0o7qe/yCqk7Y0tEhhcuSMrGn/l9N/6UjgN38TyoyvVbd1UH6BHGLdj9g5JRBKAvcGumymfiE0qS7IOnM3mN5W8gRJaEGDkhDim21Pm1Yg2GRBJ+z7C8AEy+Dz6OFvWuDsY24Gb+643D7KrmFObJ1n8Qyme/Y1bfvBkG+xdvGoBzyQrlDT12Qjkfoqb37HNrGUUD5cj2q54gyp80w=="
m2 = "hMQVVspFGh7NxCqLf7DVto3OxgDLn9n9gOCjOjEOYhi/VwZ3adFsbBL+zLZxYNdkKLNWCNRktRwpnWriEsW1uDnVt2LbxSLvjvRKbR/hyvpY+0UUZFS6wCWQjGyxUydDxQ88jNM5dY58/1nxsd04I3n3Mt97SuqwBN1+4VS3SsqtbR0GU1C7ODkPoCeGVd3PNkGHPgbT7QzMwxl63Pl3i/sp0I2/gqSnKu5CDS7e2WELz0hfiOJ2v2RvIon2EEbPwx1/6zxZlMhHuGXHNZKDtyqe6Dd+EIjpwhQFW3eH7fDIirRbaPPXAYsoypS5eFD3mIWUs4yVOH9ykkdKQ9FNwg=="

In [34]:
# extended greater common divisor
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


# modular inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

    
# pads a string with a '0' at the beginning if length is odd
def pad_even(x):
    return ('', '0')[len(x) % 2] + x


# ciphertext to integer
def cipher_b2n(c):
    c64 = base64.b64decode(c)
    tmp = ""
    for char in c64: # char is int
        tmp += pad_even(str(hex(char))[2:]) # add padded hex representation of char to tmp
    return int(tmp, 16) # convert tmp: hex string -> dec int


# integer to plaintext
def cipher_n2b(m):
    hex_m = hex(m)[2:] # remove "0x"
    return bytes.fromhex(hex_m).decode('utf-8') # bytes to ascii text conversion

In [22]:
int0 = cipher_b2n(m0)
int1 = cipher_b2n(m1)
int2 = cipher_b2n(m2)

In [23]:
c = (int0*key1.n*key2.n*modinv(key1.n*key2.n,key0.n) +
     int1*key0.n*key2.n*modinv(key0.n*key2.n,key1.n) +
     int2*key0.n*key1.n*modinv(key0.n*key1.n,key2.n))

In [24]:
c

1808677761027492404990819051428882486785803974508399540341980876053115936883173688109691129612820098931130026300708351394518246852171110392675981305214557636465256707142422904850326220464631622310238304589898304814037274339505829769990638152338410619349913788702805110454640093635806783140663857696014718069058599999832678770391063397267042251262379535761131006679485758687748161981985467313912550766978768255928182794012807392788749863440343515967113180238506546136150735660439551157914107321354613528466347258091611288906157129309638602781671078678103210660213070417264734722293903839098577729591326302517324054173747589726878310235522323053539601807418428333843150497223372772137847011104713021263737723032153959341079948333104612823779478574694879909283297223060734794563982381056151640327484769697373237142588603653840357379680613019985370210809468740916980667482496509065359833711906046797851970355565600823582609726957458397306630651731643541807441802969566495156024588314548393042875484557354

In [28]:
cmod = c % (key0.n*key1.n*key2.n)
cmod

5901951943421446155992399451709359044905575979355273287033576522270684157869846476310203592474644648033425539786240697370355743916814637392056587140979346426901745311795984228449999273045834293491846252376295742677859819007628395716044904812095114174841113876864333556131753717965426930374235531142751726342461123233986374240606888149805974108005878455547031187928447058638057991729311059827626492350075291626322692832727013483759260093525894034911962665266585315993147995234695109226082525419459465099969597326123433641003536678833488992570462644970421664306974470797647062288790366980072360721854048490932781991951323997327142219035071262368801334289661562655674528663944844277407794646186253538485787765763581387968439401021369252031538861081819555528021741089169916638894608656135317901424274697740091084448721444371326713447470713190067044717416826299278586378673004240508032724179585607070376665449953572283503782865650472934980722357259287683797012512099373498611161369168148591825480850178182

In [26]:
def find_invpow(x,n):
    """Finds the integer component of the n'th root of x,
    an integer such that y ** n <= x < (y + 1) ** n.
    """
    high = 1
    while high ** n < x:
        high *= 2
    low = high // 2
    while low < high:
        mid = (low + high) // 2
        if low < mid and mid**n < x:
            low = mid
        elif high > mid and mid**n > x:
            high = mid
        else:
            return mid
    return mid + 1

In [31]:
bar = find_invpow(cmod, 3)
bar

8388131359720046063872833100811325574570587786581033027443036972651370077226249633121066183702174905601194465162357203577717819386494559177357917520660217340214386313732889000995729043726276593413446227015142952546721646496051966772341834642570390834410121150397884781773266075473849193791775472457384434191196865262762067763768895485761779044905909446640835467821924369760638688355298154242940763320223330690939894916560502053278601601747985596894083811725949290750309782142188283705393989948406653725691406575786268881626886111264816551591864627974415487282240166658800843761054383756438874711807710508977464811552

In [32]:
print(bytes.fromhex(hex(bar)[2:]).decode('utf-8'))

Bravo! Pour valider le challenge, la clef est NoSpamWithRSA!
Well done, the key for th challenge is NoSpamWithRSA!
                                                                                                                                             
